# Байесовская оценка А/Б-тестов

&nbsp; &nbsp; *- А/Б тесты*  
&nbsp; &nbsp; *- Байесовское моделирование*  
&nbsp; &nbsp; *- Сравнение конверсий*   
&nbsp; &nbsp; *- Сравнение средних*  
&nbsp; &nbsp; *- Выручка на пользователя, заказы на платящего, средние чеки*  
&nbsp; &nbsp; *- Заключение*  
&nbsp; &nbsp; *- Ссылки*

Формулируются критерии оценки А/Б-тестов. Приводится терминология байесовского моделирования. Рассматривается сравнение конверсий. Далее - сравнение средних с помощью центральной предельной теоремы. Далее - выручка и заказы на платящего, средний чек.

# А/Б тесты  
  - Пример теста  
  - Причинная диаграмма  
  - Схема проведения  
  - Качественная картина  
  - Оценка теста  

Причинная диаграмма:

<center>
<img src="./figs/causal.png" alt="causal" width="500"/>
</center>

Картина А/Б тестов.

Значения метрик в группах определяются функциональностью приложения.  
Точные значения неизвестны.  
На основе собранных в эксперименте данных можно строить оценки точных значений.  
Пока данных мало, неопределенность в оценках большая.  
По мере набора данных точность увеличивается.  
С увеличением точности оценок растет уверенность, какая из групп лучше.  
Когда уверенность достигает достаточного значения, эксперимент можно останавливать.  

Байесовское моделирование позволяет оценить точное значение метрики по сэмлу и вероятность метрики в одной группе больше другой P(B>A).

<center>
<img src="./figs/ab_dynamics.png" alt="causal" width="500"/>
</center>

# Байесовское моделирование  

* Пример  
* Обобщение + терминология  

> Вам подарили подарок. Подарок в упаковке, и не ясно, что это. 
До того, как его развернуть, вы пытаетесь угадать, что внутри.

Эти рассуждения можно сделать количественными.  
Нужно предположить несколько вариантов подарка и для каждого оценить вероятность $P(подарок | упаковка)$. 
Это делается с помощью соотношения Байеса.

$$
P(B | A) = \frac{P(A | B) P(B)}{P(A)}
$$

В примере

$$
P(подарок_i | упаковка) = \frac{P(упаковка|подарок_i) P(подарок_i)}{P(упаковка)} = \frac{P(упаковка|подарок_i) P(подарок_i)}{\sum_i P(упаковка|подарок_i) P(подарок_i)}
$$


На этом примере видны основные этапы байесовского моделирования.  
Есть данные. Есть гипотезы, объясняющие данные.  
Для выбора одной из гипотез нужно посчитать вероятности получить данные в рамках гипотез - правдоподобия.  
Также нужны априорные вероятности каждой гипотезы.  
При комбинировании априорной вероятности гипотезы и вероятности получить данные в рамках гипотезы можно получить вероятность гипотезы при условии данных.  

> На страницу зашло 1000 человек, 100 из них нажали кнопку "Продолжить". Как выглядит распределение возможных значений конверсии? Значения конверсии для каждого пользователя можно считать одинаковыми, все возможные априорные значения конверсии равновероятными. 

Вероятность получить данные
$$
P(100/1000; \theta) = Binom(100/1000; \theta) = C^{100}_{1000} \theta^{100} (1 - \theta)^{1000-100}
$$

Плотность вероятности конверсий
$$
P(\theta; 100/1000) 
= \frac{P(100/1000; \theta) P(\theta)}{P(100/1000)}
= \frac{P(100/1000; \theta) P(\theta)}{\int d \theta P(100/1000; \theta) P(\theta)}
$$

$P(\theta)$ равномерно $P(\theta) = \mbox{Unif}(0, 1) = 1$

$$
P(\theta; 100/1000) 
= \frac{\theta^{100} (1 - \theta)^{900}}{\int d \theta (1 - \theta)^{900} \theta^{100} }
\sim \theta^{100} (1 - \theta)^{900}
= Beta(101, 901)
$$

График 

> На версию А страницы веб-сайта зашло 1000 человек, 100 нажали кнопку "Продолжить". На версию Б зашло 1000 человек, 110 нажали кнопку продолжить. С какой вероятностью конверсия страницы Б выше страницы А?

# Сравнение конверсий  
* Модель  
* Сравнение 2 групп  
* Количество правильно угаданных вариантов  

# Сравнение средних
  - ЦПТ
  - Модель
  - Сравнение двух групп
  - Количество правильно угаданных вариантов - для разных исходных распределений

# Выручка на пользователя, заказы на платящего, средние чеки
  - Модель
  - Сравнение групп
  - Кол-во угаданных вариантов

# Заключение

# Ссылки

Пациент приходит ко врачу. Симптомы - температура.   
Врач делает предположения на счет болезней - простуда, пневмония и т.д.  
Врач прикидывает, при каких болезнях эти симптомы вероятны.  
Дальше с учетом своего опыта и распространенности болезней ставит диагноз.  

Можно записать рассуждения количественно.
Для диагноза интересует $P(болезнь | симптомы)$.  
Вначале выбирается список возможных болезней и для каждой вычисляется $P(симптомы | болезнь)$.  
Распространенность болезней в регионе и опыт врача учитывает $P(болезнь)$.  

$$
P(болезнь | симптомы) \approx P(симптомы | болезнь) P(болезнь)
$$

На этом примере видны компоненты байесовского моделирования.  

Let’s walk through an example of using the **Bayesian approach** to guess what’s inside a present based on the **size of the box**.

### Setting:
You receive a box as a gift, but you don’t know what’s inside. Based on the **size of the box**, you want to update your belief about what the present might be. You have a few reasonable guesses, and you’ll use **Bayes’ Theorem** to compute the posterior probabilities of each possible item given the size of the box.

### Hypotheses (Possible Contents of the Box):
You consider three possible items that could be inside the box:

- \( H_1 \): A **book**.
- \( H_2 \): A **pair of shoes**.
- \( H_3 \): A **laptop**.

### Prior Probabilities:
Before looking at the size of the box, you have some prior beliefs about what the gift could be based on your knowledge of the person who gave you the gift. For example:

- \( P(H_1) = 0.3 \) (They often give books as gifts).
- \( P(H_2) = 0.5 \) (They’ve given shoes before).
- \( P(H_3) = 0.2 \) (They once mentioned buying you a laptop).

### Likelihoods (Based on the Size of the Box):
Now, you observe the **size of the box**, and you know how likely it is for each possible item to fit into a box of this size. Suppose the box is medium-sized. Based on that, you assign the following likelihoods:

- \( P(\text{Medium box} \mid H_1) = 0.8 \) (Books often come in medium-sized boxes).
- \( P(\text{Medium box} \mid H_2) = 0.6 \) (Shoes could fit in a medium-sized box, but not always).
- \( P(\text{Medium box} \mid H_3) = 0.3 \) (Laptops usually come in larger boxes, so this is less likely).

### Evidence (Total Probability of Observing the Box Size):
The total probability of receiving a **medium-sized box** is calculated by summing over all the possible contents:
\[
P(\text{Medium box}) = P(\text{Medium box} \mid H_1)P(H_1) + P(\text{Medium box} \mid H_2)P(H_2) + P(\text{Medium box} \mid H_3)P(H_3)
\]
\[
P(\text{Medium box}) = (0.8 \times 0.3) + (0.6 \times 0.5) + (0.3 \times 0.2) = 0.24 + 0.3 + 0.06 = 0.6
\]

### Applying Bayes’ Theorem:
Now, you can compute the **posterior probability** for each possible item using **Bayes’ Theorem**:

1. **Posterior for a Book**:
   \[
   P(H_1 \mid \text{Medium box}) = \frac{P(\text{Medium box} \mid H_1) P(H_1)}{P(\text{Medium box})} = \frac{0.8 \times 0.3}{0.6} = \frac{0.24}{0.6} = 0.4
   \]

2. **Posterior for Shoes**:
   \[
   P(H_2 \mid \text{Medium box}) = \frac{P(\text{Medium box} \mid H_2) P(H_2)}{P(\text{Medium box})} = \frac{0.6 \times 0.5}{0.6} = \frac{0.3}{0.6} = 0.5
   \]

3. **Posterior for a Laptop**:
   \[
   P(H_3 \mid \text{Medium box}) = \frac{P(\text{Medium box} \mid H_3) P(H_3)}{P(\text{Medium box})} = \frac{0.3 \times 0.2}{0.6} = \frac{0.06}{0.6} = 0.1
   \]

### Conclusion:
Based on the size of the box and your priors, you now have updated probabilities for what might be inside the box:
- \( P(\text{Book} \mid \text{Medium box}) = 0.4 \),
- \( P(\text{Shoes} \mid \text{Medium box}) = 0.5 \),
- \( P(\text{Laptop} \mid \text{Medium box}) = 0.1 \).

The most likely item is a **pair of shoes** (50% chance), followed by a **book** (40% chance). A **laptop** is the least likely (10% chance).

Аномалия в дашборде. Как выглядит процесс расследования причин?  
Вначале нужно сформулировать возможные предположения: баг в недавнем релизе, не отработали ETL-скрипты, ... .
С учетом прошлого опыта выбрать наиболее вероятное.  
Проверить.  

Можно записать рассуждения количественно.
Интересует $P(изменения | аномалия)$.  
Вначале выбирается список возможных причин и для каждой вычисляется $P(аномалия | изменения)$.  
Прошлый опыт учитывает $P(изменения)$.  

$$
P(изменения | аномалия) \approx P(аномалия | изменения) P(изменения)
$$

На этом примере видны компоненты байесовского моделирования. 

* В мешке лежат черные и белые шары, всего 1000 штук. Для оценки количества черных шаров вынимают шар, записывают цвет, возвращают обратно. За 20 повторов вынули 5 черных шаров. Сколько может быть черных шаров в мешке и каковы вероятности этих значений?

1)

При количестве черных шаров $N_B$ вероятность достать черный шар $p_B = N_B / N$.  
Вероятность вынуть 5 черных шаров в 20 экспериментах задается биномиальным распределением.

$$
P(5, 20 | N_B) = Binom(5, 20) = C^5_{20} p_B^5 p_W^{15}
$$

Возможные значения $N_B$ от 1 (мог попадаться один и тот же черный шар) до 999 (есть как минимум 1 белый).

Интересует

$$
P(N_B | 5, 20) = \frac{ P(5,20 | N_B) P(N_B)}{P(5,20)} = \frac{ P(5,20 | N_B) P(N_B)}{\sum \limits_{N_B=1}^{999} P(5,20 | N_B) P(N_B)}
$$

В знаменателе нормировочный коэффициент.

Значения $P(N_B)$ равновероятны $P(N_B) = 1/999$.

$$
P(N_B | 5, 20) 
= \frac{ P(5,20 | N_B)}{\sum \limits_{N_B=1}^{999} P(5,20 | N_B)} 
= \frac{C^5_{20} p_B^5 p_W^{15}}{\sum \limits_{N_B=1}^{999} C^5_{20} p_B^5 p_W^{15}}
= \frac{C^5_{20} N_B^5 (N - N_B)^{15}}{\sum \limits_{N_B=1}^{999} C^5_{20} N_B^5 (N-N_B)^{15}}
$$

Лучше так:
1000 шаров, достали-вернули 5, все 5 черные. Сколько всего черных?


$$
P(5 | N_B) = Binom(5, 5) = C^5_{5} p_B^5 p_W^{0} = p_B^5
$$

$$
P(N_B | 5) = \frac{ P(5 | N_B) P(N_B)}{\sum \limits_{N_B=1}^{1000} P(5 | N_B) P(N_B)}
$$

Значения $P(N_B)$ равновероятны $P(N_B) = 1/1000$.

$$
P(N_B | 5) 
= \frac{ P(5 | N_B)}{\sum \limits_{N_B=1}^{1000} P(5 | N_B)} 
= \frac{p_B^5}{\sum \limits_{N_B=1}^{1000} p_B^5}
= \frac{N_B^5}{\sum \limits_{N_B=1}^{1000} N_B^5}
$$